In [138]:
import os
import pandas as pd

In [139]:
df = pd.read_csv(os.path.join(os.getcwd(), r"AIBL-Collect.csv"))

df[["Manufacturer", "Model"]] = df["Imaging Protocol"].str.extract(
    "Manufacturer=(.+);Mfg Model=(.+)", expand=True
)

df = df.replace(
    {
        "GE MEDICAL SYSTEMS": "GE",
        "Philips Healthcare": "Philips",
        "Philips Medical Systems": "Philips",
        "SIEMENS|PixelMed": "Siemens",
        "SIEMENS": "Siemens",
    },
    regex=False,
)

df.drop(columns=["Research Group"], inplace=True)
df = df[(df.Sex.isin(["F", "M"]))]
info = pd.read_csv(os.path.join(os.getcwd(), r"aibl_pdxconv_01-Jun-2018.csv"))
info = info[info.DXCURREN.isin([1, 3])]
info = info[["DXCURREN", "RID"]].drop_duplicates()
info["DXCURREN"] = info.DXCURREN.map({1: "CN", 3: "AD"})
info.rename(columns={"RID": "Subject ID", "DXCURREN": "Research Group"}, inplace=True)

In [140]:
df = pd.merge(df, info, on="Subject ID")

In [141]:
df[(df["Subject ID"] == 2)]

,Subject ID,Sex,Visit,Age,Description,Imaging Protocol,Image ID,Manufacturer,Model,Research Group
368,2,M,Baseline,70.0,MPRAGE ADNI confirmed,Acquisition Plane=SAGITTAL;Manufacturer=SIEMEN...,450385,Siemens,TrioTim;Weighting=T1;Acquisition Type=3D,CN


In [142]:
df["Subject ID"].nunique()

600

In [143]:
df["Image ID"].nunique()

1151

In [144]:
df.groupby("Research Group")["Subject ID"].nunique()

Research Group
AD    106
CN    499
Name: Subject ID, dtype: int64

In [145]:
df.Manufacturer.unique()

array(['Siemens'], dtype=object)

In [146]:
df = df[(df["Visit"].isin(["Baseline"]))]

In [147]:
df["Image ID"].nunique()

576

In [148]:
df.groupby("Manufacturer")["Image ID"].nunique()

Manufacturer
Siemens    576
Name: Image ID, dtype: int64

In [149]:
df.groupby(["Manufacturer", "Research Group"])["Image ID"].nunique()

Manufacturer  Research Group
Siemens       AD                 96
              CN                485
Name: Image ID, dtype: int64

In [132]:
df.groupby(["Visit", "Research Group"])["Image ID"].nunique()

Visit     Research Group
Baseline  AD                 96
          CN                485
Name: Image ID, dtype: int64

In [133]:
df.groupby(["Manufacturer", "Research Group"])["Subject ID"].nunique()

Manufacturer  Research Group
Siemens       AD                 96
              CN                485
Name: Subject ID, dtype: int64

In [134]:
df.groupby(["Manufacturer", "Research Group", "Sex"])["Image ID"].nunique()

Manufacturer  Research Group  Sex
Siemens       AD              F       51
                              M       45
              CN              F      279
                              M      206
Name: Image ID, dtype: int64

In [135]:
df.groupby(["Manufacturer", "Research Group"])["Age"].mean()

Manufacturer  Research Group
Siemens       AD                73.510417
              CN                72.383505
Name: Age, dtype: float64

In [136]:
df.groupby(["Manufacturer", "Research Group"])["Age"].std()

Manufacturer  Research Group
Siemens       AD                7.490337
              CN                6.175242
Name: Age, dtype: float64